In [24]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# =============================================================================
# [작업 과정 요약]
# - AIcrowd에서 제공하는 위성 이미지 타일 및 COCO 형식 건물 어노테이션을 다운로드하여 압축을 해제함
# 링크: https://www.aicrowd.com/challenges/mapping-challenge-old
# - 각 이미지 타일별 polygon segmentation 정보를 인스턴스 분할 마스크로 변환
# - 변환된 이미지와 인스턴스 마스크를 PyTorch 학습용 .pt 파일로 각각 저장
# - 최종적으로 모델 학습에 바로 사용 가능한 image/mask 딕셔너리 파일셋을 구글 드라이브에 구축함
# - 작업 경로, 입출력 폴더, 처리 개수 등은 코드 내에서 명확히 관리됨
# =============================================================================



# 필수 라이브러리 설치 (변경 없다면 셀 한번만 실행)
!pip install aicrowd-cli geopandas rasterio torch torchvision tqdm

import os
import json
import torch
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
from rasterio import features
from tqdm import tqdm

# AIcrowd 챌린지 참가용 키 입력
API_KEY = '여기에 AIcrowd 개인 api 키 복붙해오기'

# 1. AIcrowd 로그인 (필수)
!aicrowd login --api-key {API_KEY}

# 2. 데이터셋 다운로드
!aicrowd dataset download --challenge mapping-challenge-old

# 3. 압축 해제할 작업공간 폴더 생성
!mkdir -p /content/mapping-challenge-old/train
!mkdir -p /content/mapping-challenge-old/val
!mkdir -p /content/mapping-challenge-old/test_images

# 4. 다운로드된 tar.gz 압축파일을 각각 폴더에 풀기 (경로와 파일명 주의)
!tar -xvzf /content/train.tar.gz -C /content/mapping-challenge-old/train
!tar -xvzf /content/val.tar.gz -C /content/mapping-challenge-old/val
!tar -xvzf /content/test_images.tar.gz -C /content/mapping-challenge-old/test_images

# 5. 전처리용 입력 파일 경로 설정 (실제 폴더 구조에 맞게 수정 필요)
ANNOTATION_JSON = '/content/mapping-challenge-old/train/train/annotation.json'
IMAGES_DIR = '/content/mapping-challenge-old/train/train/images'

# 6. 전처리 결과 저장 경로 (구글 드라이브에 저장: 영구적 보존)
OUTPUT_DIR = '/content/drive/MyDrive/mapping_challenge_final_dataset'
os.makedirs(os.path.join(OUTPUT_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels_dict'), exist_ok=True)

# 7. 실제 처리할 데이터 개수를 제한 (빠른 테스트/리소스 절약)
MAX_SAMPLES = 2000

# ----------------------------------------
# [헬퍼 함수]
# 라벨 마스크(인스턴스 ID로 칠해진 2D 배열)를 딥러닝 검증용 PT 포맷 타겟 딕셔너리로 변환
def convert_to_target_format(label_mask):
    # 고유 인스턴스 ID만 추출: 건물마다 1번 이상 값 (배경0 제외)
    instance_ids = torch.unique(label_mask)
    instance_ids = instance_ids[instance_ids != 0]
    if len(instance_ids) == 0:
        # 건물이 아예 없으면 빈 딕셔너리 반환 (학습 오류 예방)
        return {'instance_class': torch.tensor([], dtype=torch.long),
                'masks': torch.tensor([], dtype=torch.uint8)}
    # 각 인스턴스별 마스크 생성 [N, H, W] (N: 건물 수)
    masks = (label_mask == instance_ids[:, None, None])
    instance_classes = torch.zeros(len(instance_ids), dtype=torch.long) # 모두 '건물'
    return {'instance_class': instance_classes, 'masks': masks.to(torch.uint8)}

# ----------------------------------------
# [헬퍼 함수]
# COCO polygon 어노테이션을 실제 마스크(이미지 크기만큼의 인스턴스ID)로 변환
def polygons_to_mask(polygons, height, width):
    mask = np.zeros((height, width), dtype=np.int32)
    for idx, poly_coords in enumerate(polygons, 1):
        poly = Polygon(poly_coords)
        if not poly.is_valid:
            poly = poly.buffer(0)  # 오류 다각형 보정
        if not poly.is_empty:
            rasterized = features.rasterize([(poly, idx)], out_shape=(height, width))
            mask = np.maximum(mask, rasterized)
    return mask

# ----------------------------------------
# 8. 어노테이션 JSON 파일 로드: COCO 구조
with open(ANNOTATION_JSON, 'r') as f:
    coco = json.load(f)

# 9. 이미지의 고유ID별로 해당하는 어노테이션들 빠르게 접근 가능하도록 딕셔너리로 재구성
image_id_to_anns = {}
for ann in coco['annotations']:
    image_id_to_anns.setdefault(ann['image_id'], []).append(ann)

# ----------------------------------------
# 10. 메인 전처리 반복문: 이미지별로 마스크 생성 & 파일 저장
for img_info in tqdm(coco['images'][:MAX_SAMPLES]):   # 총 2000개만 처리
    img_path = os.path.join(IMAGES_DIR, img_info['file_name'])  # 이미지 파일 경로
    image = Image.open(img_path).convert('RGB')
    width, height = image.size

    anns = image_id_to_anns.get(img_info['id'], [])
    polygons = []
    # COCO segmentation은 여러 구조가 가능해 안전하게 추출
    for ann in anns:
        seg = ann.get('segmentation', [])
        if isinstance(seg, list) and len(seg) > 0:
            if isinstance(seg[0], list) or isinstance(seg[0], tuple):
                for poly in seg:
                    if len(poly) >= 6 and len(poly) % 2 == 0:
                        points = [(poly[i], poly[i+1]) for i in range(0, len(poly), 2)]
                        polygons.append(points)
            else:
                if len(seg) >= 6 and len(seg) % 2 == 0:
                    points = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                    polygons.append(points)

    if len(polygons) == 0:
        continue  # 어노테이션이 없으면 패스

    # 다각형 리스트를 마스크로 변환하고
    label_mask = polygons_to_mask(polygons, height, width)
    # 모델 학습용 타겟으로 변환
    target = convert_to_target_format(torch.from_numpy(label_mask).long())

    # 이미지 데이터를 (C, H, W)로 PyTorch 텐서로 바꾸고 저장
    image_tensor = torch.from_numpy(np.array(image)).permute(2,0,1).float()
    img_save_path = os.path.join(OUTPUT_DIR, 'images', f"{os.path.splitext(img_info['file_name'])[0]}.pt")
    target_save_path = os.path.join(OUTPUT_DIR, 'labels_dict', f"{os.path.splitext(img_info['file_name'])[0]}.pt")
    torch.save(image_tensor, img_save_path)
    torch.save(target, target_save_path)

print(f"총 {MAX_SAMPLES}개 샘플에 대해 데이터 전처리 및 저장 완료!")
